
 # Databases

## PETE 2061 Lab 11 Solution

## Lab 11 (Create your new .ipynb or .py file )
* Using the code given in this workbook (or the db_manipulation.py), write a Python code that loops through all given 17 wells. In each iteration of the loop:
    * Insert the Decline curve analysis model parameters into the DCAparams table in the database 
        * ** -5 if this is not done**
    * Insert the actual and model rates and cumulative production into the Rates table in the database
        * ** -5 if actual rates and cumulative production are not put in the database**
        * ** -5 if model rates and cumulative production are not put in the database**
        * ** -5 if model rates and cumulative production are wrong. This can be easily checked in the plots**
    * Plot and save a figure of the rate and cumulative production vs time (in months).
        * ** -5 if this is not done or if the plots are wrong **
<br>    
Hint: You need to use the exponential model if b = 0, and the hyperbolic model otherwise.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

import pandas as pd
import sqlite3

#create a database named "DCA.db" in the folder where this code is located
conn = sqlite3.connect("DCA.db")  #It will only connect to the DB if it already exists

#create data table to store summary info about each case/well
cur = conn.cursor()

#Custom Plot parameters
titleFontSize = 18
axisLabelFontSize = 15
axisNumFontSize = 13


dfLength = 24

for wellID in range(1,18):
    # Load spreadsheet
    fileName = 'DCA_Well ' + str(wellID) + '.xlsx'
    
    xl = pd.ExcelFile(fileName)
    
    # Load a sheet into a DataFrame by name: df1
    df1 = xl.parse('DCARegression')
    
    rateDF = pd.DataFrame({'wellID':wellID*np.ones(dfLength,dtype=int), 'time':range(1,dfLength+1),'rate':df1.iloc[8:32,1].values})
    rateDF['Cum'] = rateDF['rate'].cumsum()
    
    #insert data into the summary table
    qi = df1.iloc[2,3]
    Di = df1.iloc[3,3]
    b  = df1.iloc[4,3]
    
    cur.execute("INSERT INTO DCAparams VALUES ({},{},{},{})".format(wellID, qi, Di, b))
    conn.commit()
    
    t = np.arange(1,dfLength+1)
    Di = Di/12   #convert to monthly
    
    if b>0:
        q = 30.4375*qi/((1 + b*Di*t)**(1/b))
        Np = 30.4375*(qi/(Di*(1-b)))*(1-(1/(1+(b*Di*t))**((1-b)/b))) #30.4375 = 365.125/12
    else:
        q = qi*np.exp(-Di*t)
        Np = 30.4375*(qi-q)/Di
        q = 30.4375*q
        
    error_q = rateDF['rate'].values - q
    SSE_q = np.dot(error_q, error_q)
    
    errorNp = rateDF['Cum'].values - Np
    SSE_Np = np.dot(errorNp,errorNp)
    
    
    rateDF['q_model'] = q
    rateDF['Cum_model'] = Np
    # Use DataFrame's to_sql() function to put the dataframe into a database table called "Rates"
    rateDF.to_sql("Rates", conn, if_exists="append", index = False)


    # Read from Rates database table using the SQL SELECT statement
    prodDF = pd.read_sql_query(f"SELECT time,Cum,Cum_model FROM Rates WHERE wellID={wellID};", conn)    
    dcaDF = pd.read_sql_query("SELECT * FROM DCAparams;", conn) #this will grab everything in DCAparams table  
    

    #remaining code in loop plots the graphs
    currFig = plt.figure(figsize=(7,5), dpi=100)
    
    # Add set of axes to figure
    axes = currFig.add_axes([0.15, 0.15, 0.7, 0.7])# left, bottom, width, height (range 0 to 1)
    
    # Plot on that set of axes
    axes.plot(prodDF['time'], prodDF['Cum']/1000, color="red", ls='None', marker='o', markersize=5,label = 'well '+str(wellID) )
    axes.plot(prodDF['time'], prodDF['Cum_model']/1000, color="red", lw=3, ls='-',label = 'well '+str(wellID) )
    axes.legend(loc=4)
    axes.set_title('Cumulative Production vs Time', fontsize=titleFontSize, fontweight='bold')
    axes.set_xlabel('Time, Months', fontsize=axisLabelFontSize, fontweight='bold') # Notice the use of set_ to begin methods
    axes.set_ylabel('Cumulative Production, Mbbls', fontsize=axisLabelFontSize, fontweight='bold')
    axes.set_ylim([0, 1200])
    axes.set_xlim([0, 25])
    xticks = range(0,30,5) #np.linspace(0,4000,5)
    axes.set_xticks(xticks)
    axes.set_xticklabels(xticks, fontsize=axisNumFontSize); 
    
    yticks = [0, 400, 800, 1200]
    axes.set_yticks(yticks)
    axes.set_yticklabels(yticks, fontsize=axisNumFontSize); 
    
    currFig.savefig('well'+str(wellID)+'_Gp.png', dpi=100)




#Syntax to create a foreign key (in SQLite) in the Rates table
cur.execute("ALTER TABLE Rates RENAME TO _old_Rates;")
cur.execute("CREATE TABLE Rates                                  \
(                                                                \
  rateID INTEGER PRIMARY KEY AUTOINCREMENT,                      \
  wellID INTEGER NOT NULL,                                       \
  time INTEGER NOT NULL,                                         \
  rate REAL, \
  Cum REAL, \
  q_model REAL, \
  Cum_model REAL, \
  CONSTRAINT fk_DCAparams                                        \
    FOREIGN KEY (wellID)                                         \
    REFERENCES DCAparams (wellID)                                \
);")

cur.execute("INSERT INTO Rates (wellID, time, rate, Cum, q_model, Cum_model) \
            SELECT wellID, time, rate, Cum, q_model, Cum_model FROM _old_Rates;")
conn.commit()


conn.close()

OperationalError: no such table: DCAparams